In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from d2l import torch as d2l


In [38]:
# 定义模型
input_shape = (1, 1500, 1500)  # 输入图像的形状
num_classes = 2  # 二分类任务
batch_size = 128  # 批量大小
lr=0.05
lr=0.9
num_epochs = 500  # 训练轮数

net = nn.Sequential(
    ### 第一层卷积
    nn.Conv2d(in_channels=input_shape[0], out_channels=10, kernel_size=10, stride=5, padding=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    ### 第二层卷积
    nn.Conv2d(in_channels=10, out_channels=20, kernel_size=10, stride=5, padding=5),
    nn.ReLU(),
    nn.Dropout(0.25),
    nn.MaxPool2d(kernel_size=2, stride=2),
    # 展平
    nn.Flatten(),
    #### 全连接层
    nn.Linear(4500, 64),  # 假设输入形状为 (1, 1500, 1500)，经过卷积和池化后的形状为 (20, 15, 15)
    nn.ReLU(),
    nn.Dropout(0.5),
    
    # 输出层
    nn.Linear(64, num_classes),
    # nn.Softmax(dim=1)
)
X = torch.rand(size=(1, 1, 1500, 1500), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 10, 301, 301])
ReLU output shape: 	 torch.Size([1, 10, 301, 301])
MaxPool2d output shape: 	 torch.Size([1, 10, 150, 150])
Conv2d output shape: 	 torch.Size([1, 20, 31, 31])
ReLU output shape: 	 torch.Size([1, 20, 31, 31])
Dropout output shape: 	 torch.Size([1, 20, 31, 31])
MaxPool2d output shape: 	 torch.Size([1, 20, 15, 15])
Flatten output shape: 	 torch.Size([1, 4500])
Linear output shape: 	 torch.Size([1, 64])
ReLU output shape: 	 torch.Size([1, 64])
Dropout output shape: 	 torch.Size([1, 64])
Linear output shape: 	 torch.Size([1, 2])


In [39]:
# 数据路径前缀
PATH_PREFIX = "/root/dev/FlowPic/datasets/file_transfer_vs_all_"

# 加载数据
x_train = np.load(PATH_PREFIX + "reg_x_train.npy")
y_train_true = np.load(PATH_PREFIX + "reg_y_train.npy")
x_val = np.load(PATH_PREFIX + "reg_x_val.npy")
y_val_true = np.load(PATH_PREFIX + "reg_y_val.npy")

# 打印数据形状
print("x_train shape:", x_train.shape)  # 应为 (N, 1, 1500, 1500)
print("y_train_true shape:", y_train_true.shape)  # 应为 (N,)
print("x_val shape:", x_val.shape)  # 应为 (M, 1, 1500, 1500)
print("y_val_true shape:", y_val_true.shape)  # 应为 (M,)

# 转换为 PyTorch 张量
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train_true = torch.tensor(y_train_true, dtype=torch.int64)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val_true = torch.tensor(y_val_true, dtype=torch.int64)

# 创建数据集和数据加载器
train_dataset = TensorDataset(x_train, y_train_true)
val_dataset = TensorDataset(x_val, y_val_true)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print(f"train dataset lenght:{len(train_dataset)}, train data loader length :{len(train_loader)}")
for x_batch, y_batch in train_loader:
    print(x_batch.shape, y_batch.shape)

x_train shape: (652, 1, 1500, 1500)
y_train_true shape: (652,)
x_val shape: (73, 1, 1500, 1500)
y_val_true shape: (73,)
train dataset lenght:652, train data loader length :6
torch.Size([128, 1, 1500, 1500]) torch.Size([128])
torch.Size([128, 1, 1500, 1500]) torch.Size([128])
torch.Size([128, 1, 1500, 1500]) torch.Size([128])
torch.Size([128, 1, 1500, 1500]) torch.Size([128])
torch.Size([128, 1, 1500, 1500]) torch.Size([128])
torch.Size([12, 1, 1500, 1500]) torch.Size([12])


In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = optim.Adam(net.parameters(), lr=lr)  # Adam 优化器
# 训练参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU 或 CPU
net.to(device)  # 将模型移动到设备


# animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],legend=['train loss', 'train acc', 'test acc'])
timer, num_batches = d2l.Timer(), len(train_loader)
# 训练循环
for epoch in range(num_epochs):
    net.train()  # 设置模型为训练模式
    running_loss = 0.0
    correct = 0
    total = 0
    #创建每个epoch的指标， metric是个累加器，存储train loss  train accuracy  test accuracy
    metric = d2l.Accumulator(3)
    
    for i, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)  # 将数据移动到设备
        
        # 前向传播
        y_hat = net(X)
        loss = criterion(y_hat, y)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 统计损失和准确率
        #可以尝试参考lenet的损失指标计算方法
        
        running_loss += loss.item()
        _, predicted = torch.max(y_hat.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
        with torch.no_grad():
            # metric.add(loss * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            metric.add(loss * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
        train_loss = metric[0] / metric[2]
        train_acc = metric[1] / metric[2]

        
        # 打印训练信息
        # if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], "
        #           f"Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%")
            # print(f"d2l loss :{train_loss}, train acc:{train_acc}")
            # animator.add(epoch + (i + 1) / num_batches,(train_loss, train_acc, None))

        # # 打印训练信息
        # if (i + 1) % 10 == 0:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], "
        #           f"Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%")
    
    # 每个 epoch 结束后打印平均损失和准确率
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Average Loss: {epoch_loss:.4f}, Average Accuracy: {epoch_acc:.2f}%")
    
    # 验证模型
    net.eval()  # 设置模型为评估模式
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

Epoch [1/500], Average Loss: 0.8471, Average Accuracy: 54.91%
Validation Loss: 0.7778, Validation Accuracy: 46.58%
Epoch [2/500], Average Loss: 0.8379, Average Accuracy: 54.29%
Validation Loss: 0.7344, Validation Accuracy: 53.42%
Epoch [3/500], Average Loss: 0.7940, Average Accuracy: 46.01%
Validation Loss: 0.7001, Validation Accuracy: 46.58%
Epoch [4/500], Average Loss: 0.7544, Average Accuracy: 53.37%
Validation Loss: 0.7332, Validation Accuracy: 53.42%
Epoch [5/500], Average Loss: 0.7174, Average Accuracy: 49.08%
Validation Loss: 0.7110, Validation Accuracy: 46.58%
Epoch [6/500], Average Loss: 0.7319, Average Accuracy: 53.07%
Validation Loss: 0.7096, Validation Accuracy: 53.42%
Epoch [7/500], Average Loss: 0.6836, Average Accuracy: 57.06%
Validation Loss: 0.7100, Validation Accuracy: 46.58%
Epoch [8/500], Average Loss: 0.6911, Average Accuracy: 47.85%
Validation Loss: 0.7373, Validation Accuracy: 53.42%
Epoch [9/500], Average Loss: 0.7171, Average Accuracy: 55.83%
Validation Loss: 0